In [9]:

import pandas as pd
import glob
import os
import glob, os, re


In [10]:

# Ruta donde guardaste los archivos, ej. "./"
path = "./data_"

# Busca todos los archivos tipo "historical-ems-hourly-load-for-*-2024.xlsx"
files = sorted(glob.glob(os.path.join(path, "historical-ems-hourly-load-for-*-2024.xlsx")))
print(f"Archivos encontrados: {len(files)}")
for f in files:
    print(" -", os.path.basename(f))


Archivos encontrados: 12
 - historical-ems-hourly-load-for-01-2024.xlsx
 - historical-ems-hourly-load-for-02-2024.xlsx
 - historical-ems-hourly-load-for-03-2024.xlsx
 - historical-ems-hourly-load-for-04-2024.xlsx
 - historical-ems-hourly-load-for-05-2024.xlsx
 - historical-ems-hourly-load-for-06-2024.xlsx
 - historical-ems-hourly-load-for-07-2024.xlsx
 - historical-ems-hourly-load-for-08-2024.xlsx
 - historical-ems-hourly-load-for-09-2024.xlsx
 - historical-ems-hourly-load-for-10-2024.xlsx
 - historical-ems-hourly-load-for-11-2024.xlsx
 - historical-ems-hourly-load-for-12-2024.xlsx


In [15]:
import pandas as pd
import glob, os, re

# ==============================================================
# 1️⃣ Configuración
# ==============================================================
path = "./data_/"
utility = "SDGE"  # Cambia a "PGE", "SCE", "CAISO" si lo deseas

# ==============================================================
# 2️⃣ Lectura y limpieza flexible
# ==============================================================
files = sorted(glob.glob(os.path.join(path, "historical-ems-hourly-load-for-*-2024.xlsx")))
print(f"Archivos encontrados: {len(files)}")

dfs = []
for f in files:
    print("Leyendo:", os.path.basename(f))
    # Leer con separador decimal correcto
    df = pd.read_excel(f, header=0, decimal=",")
    df.columns = [c.strip().upper() for c in df.columns]

    # Detectar columna de fecha
    date_col = next((c for c in df.columns if "DATE" in c), None)
    if date_col is None:
        print("⚠️ No se encontró columna de fecha en", os.path.basename(f))
        continue

    # Convertir fecha
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col])

    # Estandarizar nombres
    df.columns = [re.sub(r"[^A-Z]", "", c.upper()) for c in df.columns]

    # Detectar y renombrar columna de SDGE
    if "SDGE" not in df.columns:
        alt = next((c for c in df.columns if "SDG" in c), None)
        if alt:
            df.rename(columns={alt: "SDGE"}, inplace=True)

    # Detectar y renombrar columna de hora (HR)
    if "HR" not in df.columns:
        alt_hr = next((c for c in df.columns if c.startswith("H")), None)
        if alt_hr:
            df.rename(columns={alt_hr: "HR"}, inplace=True)

    # Renombrar columna de fecha uniformemente
    df.rename(columns={date_col: "DATE"}, inplace=True)

    dfs.append(df)
    print(f"✅ {len(df)} filas válidas")

# Combinar todo
load_all = pd.concat(dfs, ignore_index=True)
load_all[utility] = pd.to_numeric(load_all[utility], errors="coerce")

# ==============================================================
# 3️⃣ Dataset horario (formato NYC extendido)
# ==============================================================
hourly = load_all.copy()
hourly["datetime"] = pd.to_datetime(hourly["DATE"]) + pd.to_timedelta(hourly["HR"] - 1, unit="h")
hourly = hourly.sort_values("datetime").reset_index(drop=True)

hourly["meter_id"] = f"{utility}_EMS.T"
hourly.rename(columns={utility: "reading"}, inplace=True)
hourly = hourly[["meter_id", "datetime", "reading"]]

hourly_out = f"./{utility}_2024_hourly.csv"
hourly.to_csv(hourly_out, index=False)
print(f"✅ Dataset horario guardado: {hourly_out}")

# ==============================================================
# 4️⃣ Dataset diario (agregado)
# ==========================================


Archivos encontrados: 12
Leyendo: historical-ems-hourly-load-for-01-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-02-2024.xlsx
✅ 696 filas válidas
Leyendo: historical-ems-hourly-load-for-03-2024.xlsx
✅ 743 filas válidas
Leyendo: historical-ems-hourly-load-for-04-2024.xlsx
✅ 720 filas válidas
Leyendo: historical-ems-hourly-load-for-05-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-06-2024.xlsx
✅ 720 filas válidas
Leyendo: historical-ems-hourly-load-for-07-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-08-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-09-2024.xlsx
✅ 720 filas válidas
Leyendo: historical-ems-hourly-load-for-10-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-11-2024.xlsx
✅ 721 filas válidas
Leyendo: historical-ems-hourly-load-for-12-2024.xlsx
✅ 744 filas válidas
✅ Dataset horario guardado: ./SDGE_2024_hourly.csv


In [16]:
import pandas as pd
import glob, os, re

# ==============================================================
# 1️⃣ Configuración
# ==============================================================
utility = "SDGE"  # o "PGE", "SCE", "CAISO"

# ==============================================================
# 2️⃣ Lectura y limpieza
# ==============================================================
files = sorted(glob.glob(os.path.join(path, "historical-ems-hourly-load-for-*-2024.xlsx")))
print(f"Archivos encontrados: {len(files)}")

dfs = []
for f in files:
    print("Leyendo:", os.path.basename(f))
    df = pd.read_excel(f, header=0, decimal=",")
    df.columns = [c.strip().upper() for c in df.columns]

    date_col = next((c for c in df.columns if "DATE" in c), None)
    if date_col is None:
        print("⚠️ No se encontró columna de fecha en", os.path.basename(f))
        continue

    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col])
    df.columns = [re.sub(r"[^A-Z]", "", c.upper()) for c in df.columns]

    if "SDGE" not in df.columns:
        alt = next((c for c in df.columns if "SDG" in c), None)
        if alt:
            df.rename(columns={alt: "SDGE"}, inplace=True)

    if "HR" not in df.columns:
        alt_hr = next((c for c in df.columns if c.startswith("H")), None)
        if alt_hr:
            df.rename(columns={alt_hr: "HR"}, inplace=True)

    df.rename(columns={date_col: "DATE"}, inplace=True)
    dfs.append(df)
    print(f"✅ {len(df)} filas válidas")

load_all = pd.concat(dfs, ignore_index=True)
load_all[utility] = pd.to_numeric(load_all[utility], errors="coerce")

# ==============================================================
# 3️⃣ Cálculo mensual y orden cronológico
# ==============================================================
monthly_sum = (
    load_all.groupby(pd.to_datetime(load_all["DATE"]).dt.to_period("M"))[utility]
    .sum()
    .reset_index()
)

# Convertir Period → Timestamp (primer día de cada mes)
monthly_sum["DATE"] = monthly_sum["DATE"].dt.to_timestamp()
monthly_sum = monthly_sum.rename(columns={utility: "reading"})

monthly_sum = monthly_sum.sort_values("DATE").reset_index(drop=True)
monthly_sum["meter_id"] = f"{utility}_EMS.T"
monthly_sum["date"] = monthly_sum["DATE"].dt.strftime("%Y-%m-%d")
monthly_sum = monthly_sum[["meter_id", "date", "reading"]]

# ==============================================================
# 4️⃣ Guardar CSV final mensual
# ==============================================================
out_file = f"./{utility}_2024_monthly.csv"
monthly_sum.to_csv(out_file, index=False)

print(f"\n✅ Archivo mensual generado y ordenado cronológicamente: {out_file}")
print(monthly_sum.head(5))


Archivos encontrados: 12
Leyendo: historical-ems-hourly-load-for-01-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-02-2024.xlsx
✅ 696 filas válidas
Leyendo: historical-ems-hourly-load-for-03-2024.xlsx
✅ 743 filas válidas
Leyendo: historical-ems-hourly-load-for-04-2024.xlsx
✅ 720 filas válidas
Leyendo: historical-ems-hourly-load-for-05-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-06-2024.xlsx
✅ 720 filas válidas
Leyendo: historical-ems-hourly-load-for-07-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-08-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-09-2024.xlsx
✅ 720 filas válidas
Leyendo: historical-ems-hourly-load-for-10-2024.xlsx
✅ 744 filas válidas
Leyendo: historical-ems-hourly-load-for-11-2024.xlsx
✅ 721 filas válidas
Leyendo: historical-ems-hourly-load-for-12-2024.xlsx
✅ 744 filas válidas

✅ Archivo mensual generado y ordenado cronológicamente: ./SDGE_2024_monthly.csv
     meter_id     